In [1]:
import torch
import numpy as np
from  tqdm import tqdm


In [2]:
with open("../Data/mobydick.txt","r",encoding='utf-8') as file:
    raw_txt = file.read()

In [3]:
# Not beautiful
stripped_txt = raw_txt.replace('\n\n', ' ').replace('\t', ' ').replace('\r', ' ')\
    .replace(';', '').replace(':', '').replace('  ', ' ')

In [4]:
vocab = list(set(stripped_txt))
vocab.sort()
vocab_dict = {}
for i, character in enumerate(vocab):
    identity_vector = torch.zeros(len(vocab))
    identity_vector[i] = 1
    vocab_dict[character] = identity_vector

In [5]:
# L
L = 15
dataX = torch.zeros((len(stripped_txt) - L, L, len(vocab_dict)))
dataY = torch.zeros(len(stripped_txt) - L)
for i in range(len(stripped_txt) - L):
    x = torch.zeros((L, len(vocab_dict)))
    for j in range(L):
        x[j] = vocab_dict[stripped_txt[i+j]]
    dataX[i] = x
    dataY[i] = torch.argmax(vocab_dict[stripped_txt[i+L]])

In [6]:
reversed_dict = {}
for key in vocab_dict:
    reversed_dict[torch.argmax(vocab_dict[key]).item()] = key
for x in dataX[0]:
    print(reversed_dict[torch.argmax(x).item()])
    print(x)
print(reversed_dict[dataY[0].item()])
print(dataY[0])

﻿
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.])
C
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
H
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [7]:
from torch.utils.data import Dataset


class MobyDickDataset(Dataset):

    def __init__(self, x, y):
        self.X = x
        self.Y = y

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.Y[idx]
        return (x, y)

    def __len__(self):
        return self.X.shape[0]

In [9]:
dataset = MobyDickDataset(dataX, dataY)
print(len(dataset))
train_set, val_set = torch.utils.data.random_split(dataset, [1170000, 14818])

1184818


In [10]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=256, shuffle=True, num_workers=0)

In [11]:
class MyLSTM(torch.nn.Module):


    def __init__(self, feature_size, hidden_size, nr_of_stacks, output_dim):
        super().__init__()
        self.LSTM = torch.nn.LSTM(feature_size, hidden_size, nr_of_stacks, batch_first=True)
        self.linear = torch.nn.Linear(hidden_size, output_dim)
        self.hidden_size = hidden_size

    def forward(self, x):
        out, (h, c) = self.LSTM(x)
        out = self.linear(h)
        out = out[0]
        return out

In [12]:
import datetime

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Device {device}.")

def train(model, optimizer, loss_f, epochs, train_loader):
    model.train()
    for epoch in range(epochs):
        loss_train = 0.0
        train_bar = tqdm(total=len(train_loader), desc="Epoch progression")

        for xb, yb in train_loader:
            xb = xb.to(device=device)
            yb = yb.to(device=device)
            yb = yb.long()
            with torch.cuda.amp.autocast(enabled=True):
                out = model(xb)
                loss = loss_f(out, yb)
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()
                loss_train += loss.item()
                train_bar.update()

        print('{}  |  Epoch {}  |  Training loss {:.3f}'.format(
            datetime.datetime.now(), epoch,
            loss_train / len(train_loader)))
        train_bar.close()

Device cuda.


In [13]:
model = MyLSTM(len(vocab_dict), len(vocab_dict), 1, len(vocab_dict))
opt = torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()
model.to(device=device)
loss.to(device=device)
train(model, opt, loss, 55, train_loader)

Epoch progression:   1%|          | 66/9141 [00:00<00:26, 337.50it/s]

2021-04-28 20:35:24.566174  |  Epoch 0  |  Training loss 2.180


Epoch progression:   1%|          | 66/9141 [00:00<00:26, 336.91it/s]

2021-04-28 20:35:47.440746  |  Epoch 1  |  Training loss 1.838


Epoch progression:   1%|          | 68/9141 [00:00<00:26, 347.41it/s]

...

2021-04-28 20:55:13.808017  |  Epoch 52  |  Training loss 1.440


Epoch progression:   0%|          | 27/9141 [00:00<00:33, 269.76it/s]

2021-04-28 20:55:36.821924  |  Epoch 53  |  Training loss 1.440


Epoch progression: 100%|██████████| 9141/9141 [00:22<00:00, 404.16it/s]

2021-04-28 20:55:59.439472  |  Epoch 54  |  Training loss 1.439


In [14]:
def validate(model, val_loader, train_loader):
    """
    Plot training and validation accuracy
    """
    model.eval()
    accdict = {}
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():
            for imgs, labels in loader:
                imgs = imgs.to(device=device)
                labels = labels.to(device=device)

                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1)
                total += labels.shape[0]
                correct += int((predicted == labels).sum())

        print("Accuracy {}: {:.2f}".format(name , correct / total))
        accdict[name] = correct / total
    return accdict

In [15]:
validate(model, val_loader, train_loader)

Accuracy train: 0.57
Accuracy val: 0.54


{'train': 0.5667504273504274, 'val': 0.544270481846403}

In [16]:
def generate_text(model, seed):
    seed = seed.to(device)
    for i, t in enumerate(seq_seed_tensor[0]):
        print(reversed_dict[torch.argmax(t).item()], end='')

    for _ in range(500):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0,:-1] = seed[0,1:].clone()
        seed[0, -1] = vocab_dict[reversed_dict[torch.argmax(next_c).item()]]
        print(reversed_dict[torch.argmax(next_c).item()], end='')

seq_seed = "There now is yo"

seq_seed_tensor = torch.zeros((1, L, len(vocab_dict)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab_dict[c]


for i, t in enumerate(seq_seed_tensor[0]):
    print(reversed_dict[torch.argmax(t).item()], end='')

There now is yo

In [17]:
generate_text(model, seq_seed_tensor)

There now is you would have been the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was the ship was t

It repeats!

In [18]:
def generate_text_with_sampling(model, seed):
    seed = seed.to(device)
    for i, t in enumerate(seq_seed_tensor[0]):
        print(reversed_dict[torch.argmax(t).item()], end='')

    for _ in range(1000):
        next_c = model(seed)[0]
        next_c = torch.softmax(next_c, -1, next_c.dtype)
        seed[0,:-1] = seed[0,1:].clone()
        test_c = np.random.choice(len(vocab_dict), 1, p=next_c.cpu().detach().numpy())
        seed[0, -1] = vocab_dict[reversed_dict[test_c[0]]]
        print(reversed_dict[test_c[0]], end='')

In [19]:
generate_text_with_sampling(model, seq_seed_tensor)

There now is you will be in some boat to for a boy,
he bigst into something but hands at the found and therefore at that crencureless from old. Those
mates and sun in—an a
beaches, so that the God
enchore emespiant
crosch, who no bread,
maked, Sea, rises her unsure cubshement
about, shele, and
Whale more formed the whale, my underturate heard think,
assume it here and spap, to jolly assumed it like that rolled prehigets that the ship flowys, so dark in seemed to, and the imoced and corress. The far hunl he, beneath themote anoneefian
again business something has intense after the Tagesticen! Oh. “D’s see a chaps, like an old
seddells. “ How in self a few rolls, is a breastnish in
strived linet—what was no me will wake, a bitness. What Boltallousest practiousings but from a may provinted, his wonder that the
joke. Parrieful water, was wherean, and they by as three far Port Elijan circumpious round? *. Olard fience of those mark for the assurates, he alm, with sign of iron aye, Stubb wil

In [20]:
train(model, opt, loss, 25, train_loader)

Epoch progression:   0%|          | 27/9141 [00:00<00:34, 267.09it/s]

2021-04-28 20:56:34.259047  |  Epoch 0  |  Training loss 1.438


Epoch progression:   0%|          | 27/9141 [00:00<00:33, 269.76it/s]

2021-04-28 20:56:57.155343  |  Epoch 1  |  Training loss 1.438


Epoch progression:   0%|          | 25/9141 [00:00<00:37, 244.88it/s]

...

2021-04-28 21:05:14.713899  |  Epoch 22  |  Training loss 1.428


Epoch progression:   1%|          | 65/9141 [00:00<00:27, 334.20it/s]

2021-04-28 21:05:38.437222  |  Epoch 23  |  Training loss 1.428


Epoch progression: 100%|██████████| 9141/9141 [00:23<00:00, 385.49it/s]

2021-04-28 21:06:02.150744  |  Epoch 24  |  Training loss 1.428


In [21]:
seq_seed = "Call me Ishmael"

seq_seed_tensor = torch.zeros((1, L, len(vocab_dict)))
for i, c in enumerate(seq_seed):
    seq_seed_tensor[0, i] = vocab_dict[c]

generate_text_with_sampling(model, seq_seed_tensor)

Call me Ishmael! The arm, are say to intwered and sailed Pacifish bear those straits of and thundened to a
Sporoge whalemen’t the head of De
Watch, Can’t seeking blows of the
supernations in his jet pump. Like largest undievoly view
jeventable firm, of all the obseacated it by the captain’s own of that gave.” In a lowerish
would stay itself present and lannerest wife arigaling of perchp to the circumstance of the sign of than?—The matisfifiter, is all the deck. Surgeable light a brarks these fishery that indeed, especially ene fairy, he killy transpeated to me now is I looking over
eyes
were being swift all. “Where ars to
him, he crys. So sim.” I see yet were there of other stopping power woe! Captain Ahab still nighing
volumes was lightring or very procied back to his own. Maste to them that
ever in stornch of very injusingly born’s and the other sun-look for the scotchly befulled on angular secrible. The mouth that starboard, and becand Captain and has, yet not blackmes of _lie’s fig

In [22]:
validate(model, val_loader, train_loader)



Accuracy train: 0.57
Accuracy val: 0.55


{'train': 0.5697068376068376, 'val': 0.5455527061681739}